### Check GPU availability

In [1]:
!nvidia-smi

Sun Jan 20 01:19:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   27C    P8    17W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 29%   45C    P8    18W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### Import libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import time
import json
import plotly
import logging
logging.getLogger().setLevel(logging.INFO)

from pprint import pprint
from tqdm import tqdm_notebook
from idst_util import trivial
from idst_util import dstc2

from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.layout import Margin
plotly.offline.init_notebook_mode(connected = True)

[nltk_data] Downloading package punkt to /home/is/andrei-
[nltk_data]     cc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Check DSTC2 availability

In [3]:
trivial.print_idst()
dstc2.check()

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!


### Retrieve data

In [4]:
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_Y_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|       Dataset Retrieval        |
INFO:root:+--------------------------------+
INFO:root:Reading dstc2_train.flist, dstc2_dev.flist and ontology_dstc2.json
INFO:root:Asserted 1612 dialogs for dstc2_train.flist
INFO:root:Asserted 506 dialogs for dstc2_dev.flist
INFO:root:Extracting raw train features
100%|██████████| 1612/1612 [00:16<00:00, 99.16it/s] 
INFO:root:Extracting raw dev features
100%|██████████| 506/506 [00:04<00:00, 108.27it/s]
INFO:root:Reading dstc2_test.flist
INFO:root:Asserted 1117 dialogs for dstc2_test.flist
INFO:root:Extracting raw test features
100%|██████████| 1117/1117 [00:10<00:00, 103.35it/s]


### Set device

In [5]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 0
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU {}".format(GPU_ID))

INFO:root:+--------------------------------+
INFO:root:|            Baseline            |
INFO:root:+--------------------------------+
INFO:root:Running on GPU 0


### Create vocabularies

In [6]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}

for raw_train_dialog in tqdm_notebook(raw_X_train):
    
    for raw_train_turn in raw_train_dialog["turns"]:
        
        tokens_scores = raw_train_turn["system"] + raw_train_turn["user"]
        
        for token_score in tokens_scores:
            token = token_score[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries


### Execution configuration

In [7]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
logging.info("VOCABULARY_SIZE:\t\t{}".format(VOCABULARY_SIZE))

# NOTE: +2 because of null and dontcare
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2 
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2
METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = len(ontology["requestable"])
EMBEDDING_DIM = 170
ALTERED_EMBEDDING_DIM = 300
HIDDEN_DIM = 100
NUM_EPOCHS = 30
GOAL_LOSS_FUNCTION = nn.CrossEntropyLoss()
METHOD_LOSS_FUNCTION = nn.CrossEntropyLoss()
REQUESTED_LOSS_FUNCTION = nn.BCELoss()
logging.info("GOAL_FOOD_DIM:\t\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t\t{}".format(GOAL_AREA_DIM))
logging.info("METHOD_DIM:\t\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t\t{}".format(REQUESTED_DIM))
logging.info("EMBEDDING_DIM:\t\t{}".format(EMBEDDING_DIM))
logging.info("ALTERED_EMBEDDING_DIM:\t{}".format(ALTERED_EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_EPOCHS:\t\t\t{}".format(NUM_EPOCHS))
logging.info("GOAL_LOSS_FUNCTION:\t\t{}".format(GOAL_LOSS_FUNCTION))
logging.info("METHOD_LOSS_FUNCTION:\t\t{}".format(METHOD_LOSS_FUNCTION))
logging.info("REQUESTED_LOSS_FUNCTION:\t{}".format(REQUESTED_LOSS_FUNCTION))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:		1149
INFO:root:GOAL_FOOD_DIM:		93
INFO:root:GOAL_PRICERANGE_DIM:		5
INFO:root:GOAL_NAME_DIM:		115
INFO:root:GOAL_AREA_DIM:		7
INFO:root:METHOD_DIM:			5
INFO:root:REQUESTED_DIM:		8
INFO:root:EMBEDDING_DIM:		170
INFO:root:ALTERED_EMBEDDING_DIM:	300
INFO:root:HIDDEN_DIM:			100
INFO:root:NUM_EPOCHS:			30
INFO:root:GOAL_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:METHOD_LOSS_FUNCTION:		CrossEntropyLoss()
INFO:root:REQUESTED_LOSS_FUNCTION:	BCELoss()


### Models utilities

In [8]:
def get_index_and_score(turn, token_to_index, mode, device):
    indices = []
    scores = []
    if mode == "train":
        for system_token, system_token_score in turn["system"]:
            indices.append(token_to_index[system_token])
            scores.append(system_token_score)
        for user_token, user_token_score in turn["user"]:
            if np.random.binomial(n = 1, p = 0.1) == 1:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[user_token])
            scores.append(user_token_score)
    else:
        tokens_scores = turn["system"] + turn["user"]
        for token, score in tokens_scores:
            if token not in token_to_index:
                indices.append(token_to_index["<unk>"])
            else:
                indices.append(token_to_index[token])
            scores.append(score)
    assert len(indices) == len(scores)
    return torch.tensor(indices, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

def plotly_plot(dev_accuracies, test_accuracies, accuracy_type):
    plotly.offline.iplot({
                            "data": [Scatter(
                                            x = list(range(len(dev_accuracies))),
                                            y = dev_accuracies,
                                            mode = "lines+markers",
                                            name = "Dev {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#3498db")),
                                    Scatter(
                                            x = list(range(len(test_accuracies))),
                                            y = test_accuracies,
                                            mode = "lines+markers",
                                            name = "Test {} Accuracy".format(accuracy_type),
                                            marker = dict(color = "#9b59b6"))],
                            "layout": Layout(
                                             title = "<b>Dev-Test {} Accuracy</b>".format(accuracy_type),
                                             xaxis = dict(title = "<b>Epoch</b>",
                                                          dtick = 1,
                                                          titlefont = dict(color = "#34495e")),
                                             yaxis = dict(title = "<b>Accuracy</b>",
                                                          titlefont = dict(color = "#34495e")),
                                             margin = Margin(b = 150))
                        })

class EarlyStopping():
    
    def __init__(self, min_delta = 0, patience = 0):
        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training

get_bin = lambda x, n: format(x, "b").zfill(n)

def make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    
    model_GoalPricerange = model_GoalPricerange.eval()
    model_GoalArea = model_GoalArea.eval()
    model_GoalName = model_GoalName.eval()
    model_GoalFood = model_GoalFood.eval()
    model_Requested = model_Requested.eval()
    model_Method = model_Method.eval()
    
    with torch.no_grad():
        tracker_json = {}
        tracker_json["dataset"] = dataset
        tracker_json["sessions"] = []

        start_time = time.time()
        for raw_X_dialog, raw_Y_dialog in tqdm_notebook(zip(raw_X, raw_Y), total = len(raw_X)):
            
            model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
            model_GoalArea.hidden = model_GoalArea.init_hidden()
            model_GoalName.hidden = model_GoalName.init_hidden()
            model_GoalFood.hidden = model_GoalFood.init_hidden()
            model_Requested.hidden = model_Requested.init_hidden()
            model_Method.hidden = model_Method.init_hidden()
            
            session = {}
            session["session-id"] = raw_X_dialog["session-id"]
            session["turns"] = []

            for raw_X_turn, raw_Y_turn in zip(raw_X_dialog["turns"], raw_Y_dialog["turns"]):
                turn = {}
                turn["goal-labels"] = {}

                indices, scores = get_index_and_score(raw_X_turn, token_to_index, mode = "eval", device = DEVICE)
                
                goal_food = model_GoalFood(indices, scores)
                goal_pricerange = model_GoalPricerange(indices, scores)
                goal_name = model_GoalName(indices, scores)
                goal_area = model_GoalArea(indices, scores)
                requested = model_Requested(indices, scores)
                method = model_Method(indices, scores)
                turn["goal-labels"]["food"] = retrieve_output_GoalFood(goal_food, ontology)
                turn["goal-labels"]["pricerange"] = retrieve_output_GoalPricerange(goal_pricerange, ontology)
                turn["goal-labels"]["name"] = retrieve_output_GoalName(goal_name, ontology)
                turn["goal-labels"]["area"] = retrieve_output_GoalArea(goal_area, ontology)
                turn["requested-slots"] = retrieve_output_Requested(requested, ontology)
                turn["method-label"] = retrieve_output_Method(method, ontology)
                
                session["turns"].append(turn)
            tracker_json["sessions"].append(session)
        end_time = time.time()
        tracker_json["wall-time"] = end_time - start_time
        
        return tracker_json

def get_scores(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset):
    goal_accuracy = 0
    goal_l2 = 0
    requested_accuracy = 0
    requested_l2 = 0
    method_accuracy = 0
    method_l2 = 0
    
    tracker = make_tracker(model_GoalPricerange, model_GoalArea, model_GoalName, model_GoalFood, model_Requested, model_Method, raw_X, raw_Y, dataset = dataset)
    
    with open("tracker_panfmr.json", "w") as tracker_file:
        json.dump(tracker, tracker_file)
    
    if dataset == "dstc2_train":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_train\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr.json\
        --scorefile tracker_panfmr.score.csv
    elif dataset == "dstc2_dev":
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_dev\
        --dataroot dstc2/dstc2_traindev/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr.json\
        --scorefile tracker_panfmr.score.csv
    else:
        !python2 dstc2/dstc2_scripts/score.py\
        --dataset dstc2_test\
        --dataroot dstc2/dstc2_test/data\
        --ontology dstc2/dstc2_scripts/config/ontology_dstc2.json\
        --trackfile tracker_panfmr.json\
        --scorefile tracker_panfmr.score.csv

    file_cat = !python2 dstc2/dstc2_scripts/report.py --scorefile tracker_panfmr.score.csv
    
    found_accuracies = False
    for line in file_cat:
        if line.startswith("Accuracy"):
            accuracies = line.split("|")
            goal_accuracy = float(accuracies[1])
            requested_accuracy = float(accuracies[2])
            method_accuracy = float(accuracies[3])
            found_accuracies = True
        if found_accuracies and line.startswith("l2"):
            l2s = line.split("|")
            goal_l2 = float(l2s[1])
            method_l2 = float(l2s[2])
            requested_l2 = float(l2s[3])
    
    return goal_accuracy, goal_l2, requested_accuracy, requested_l2, method_accuracy, method_l2

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_output_GoalFood(output_tensor, ontology):
    ontology_informable_food = ontology["informable"]["food"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_food_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_food_dict["dontcare"] = output_tensor[1].item() 
    for index in range(len(output_tensor) - 2):
        goal_food_dict[ontology_informable_food[index]] = output_tensor[index + 2].item()
    return goal_food_dict

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_output_GoalPricerange(output_tensor, ontology):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_pricerange_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_pricerange_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):     
        goal_pricerange_dict[ontology_informable_pricerange[index]] = output_tensor[index + 2].item()
    return goal_pricerange_dict

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_output_GoalName(output_tensor, ontology):
    ontology_informable_name = ontology["informable"]["name"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_name_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_name_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_name_dict[ontology_informable_name[index]] = output_tensor[index + 2].item()
        
    return goal_name_dict

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

def retrieve_output_GoalArea(output_tensor, ontology):
    ontology_informable_area = ontology["informable"]["area"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    goal_area_dict = {}
    
    index = torch.argmax(output_tensor).item()
    goal_area_dict["dontcare"] = output_tensor[1].item()
    for index in range(len(output_tensor) - 2):
        goal_area_dict[ontology_informable_area[index]] = output_tensor[index + 2].item()
        
    return goal_area_dict

def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_method = ontology["method"]
    raw_gold_method = raw_Y["method"]
    gold_method = ontology_method.index(raw_gold_method)
    return torch.tensor([gold_method], dtype = torch.long, device = device)

def retrieve_output_Method(output_tensor, ontology):
    ontology_method = ontology["method"]
    output_tensor = output_tensor.view(-1)
    output_tensor = torch.exp(output_tensor)
    method_dict = {}
    
    for index in range(len(output_tensor)):
        method_dict[ontology_method[index]] = output_tensor[index].item()
    
    return method_dict

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(len(ontology_requestable), dtype = float)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            gold_requested[ontology_requestable.index(requested)] = 1.0
    return torch.tensor([gold_requested], dtype = torch.float, device = device)

def retrieve_output_Requested(output_tensor, ontology):
    ontology_requestable = ontology["requestable"]
    output_tensor = output_tensor.view(-1)
    requested_dict = {}
    for index in range(len(output_tensor)):
        probability_value = output_tensor[index].item()
        if np.greater_equal(probability_value, 0.5):
            requested_dict[ontology_requestable[index]] = probability_value
    return requested_dict

### Goal Pricerange Model

In [9]:
class GoalPricerangeModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_pricerange_dim,
                 device):
        
        super(GoalPricerangeModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_pricerange_dim = goal_pricerange_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_pricerange_classifier = nn.Linear(in_features = hidden_dim,
                                                    out_features = goal_pricerange_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_pricerange = F.log_softmax(self.goal_pricerange_classifier(self.hidden[0]).view(-1, self.goal_pricerange_dim), dim = 1) 
        
        return goal_pricerange

model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)

optimizer_GoalPricerangeModel = optim.Adam(model_GoalPricerange.parameters(), lr = 1e-4)

### Goal Area Model

In [10]:
class GoalAreaModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_area_dim,
                 device):
        
        super(GoalAreaModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_area_dim = goal_area_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_area_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_area_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_area = F.log_softmax(self.goal_area_classifier(self.hidden[0]).view(-1, self.goal_area_dim), dim = 1)
        
        return goal_area

model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)

optimizer_GoalAreaModel = optim.Adam(model_GoalArea.parameters(), lr = 1e-4)

### Goal Name Model

In [11]:
class GoalNameModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_name_dim,
                 device):
        
        super(GoalNameModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_name_dim = goal_name_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_name_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_name_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_name = F.log_softmax(self.goal_name_classifier(self.hidden[0]).view(-1, self.goal_name_dim), dim = 1)
        
        return goal_name

model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)

optimizer_GoalNameModel = optim.Adam(model_GoalName.parameters(), lr = 1e-4)

### Goal Food Model

In [12]:
class GoalFoodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 goal_food_dim,
                 device):
        
        super(GoalFoodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.goal_food_dim = goal_food_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.goal_food_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = goal_food_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        goal_food = F.log_softmax(self.goal_food_classifier(self.hidden[0]).view(-1, self.goal_food_dim), dim = 1)        
        
        return goal_food

model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)

optimizer_GoalFoodModel = optim.Adam(model_GoalFood.parameters(), lr = 1e-4)

### Requested Model

In [13]:
class RequestedModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 requested_dim,
                 device):
        
        super(RequestedModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.requested_dim = requested_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.requested_classifier = nn.Linear(in_features = hidden_dim,
                                              out_features = requested_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        requested = torch.sigmoid(self.requested_classifier(self.hidden[0]).view(-1, self.requested_dim))
        
        return requested

model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)

optimizer_RequestedModel = optim.Adam(model_Requested.parameters(), lr = 1e-4)

### Method Model

In [14]:
class MethodModel(nn.Module):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dim,
                 altered_embedding_dim,
                 hidden_dim,
                 method_dim,
                 device):
        
        super(MethodModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.method_dim = method_dim
        self.device = device
        
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                       embedding_dim = embedding_dim)
        
        self.altered_embeddings = nn.Linear(in_features = (embedding_dim + 1),
                                            out_features = altered_embedding_dim)
        
        self.lstm = nn.LSTM(input_size = altered_embedding_dim,
                            hidden_size = hidden_dim)
        
        self.method_classifier = nn.Linear(in_features = hidden_dim,
                                           out_features = method_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indices, scores):
    
        embeddings = self.embeddings(indices)
        
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        
        altered_embeddings = F.relu(self.altered_embeddings(embeddings_concat_score))
        
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indices), 1, -1), self.hidden)
        
        method = F.log_softmax(self.method_classifier(self.hidden[0]).view(-1, self.method_dim), dim = 1)
        
        return method

model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)

optimizer_MethodModel = optim.Adam(model_Method.parameters(), lr = 1e-4)

### Train Goal Pricerange Model

In [19]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_pricerange_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalPricerange = model_GoalPricerange.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalPricerange.hidden = model_GoalPricerange.init_hidden()
                
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalPricerangeModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_pricerange = model_GoalPricerange(indices, scores)
            
            loss_goal_pricerange = GOAL_LOSS_FUNCTION(goal_pricerange,
                                                      retrieve_gold_GoalPriceRange(raw_Y_train_turn,
                                                                                   ontology = ontology,
                                                                                   device = DEVICE))
            loss_goal_pricerange.backward(retain_graph = True)
            
            optimizer_GoalPricerangeModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    goal_pricerange_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_pricerange_early_stopping.wait == 0:
        torch.save(model_GoalPricerange.state_dict(), "model_PANFMR_GoalPricerange.pt")
    
    if goal_pricerange_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.0331074(0.03)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9887474(0.99)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.0268346(0.03)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.991488(0.99)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.032586(0.03)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9886848(0.99)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.0269378(0.03)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9911362(0.99)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	3/30


INFO:root:DEV Acc:		0.0331074(0.03)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9885967(0.99)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.0271442(0.03)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9911907(0.99)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	4/30


INFO:root:DEV Acc:		0.0331074(0.03)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.988693(0.99)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.0272474(0.03)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9914716(0.99)		0.800123(0.8)		0.9121297(0.91)


### Load Goal Pricerange Model

In [20]:
model_GoalPricerange = GoalPricerangeModel(vocabulary_size = VOCABULARY_SIZE,
                                           embedding_dim = EMBEDDING_DIM,
                                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                           hidden_dim = HIDDEN_DIM,
                                           goal_pricerange_dim = GOAL_PRICERANGE_DIM,
                                           device = DEVICE)

model_GoalPricerange = model_GoalPricerange.to(DEVICE)
model_GoalPricerange.load_state_dict(torch.load("model_PANFMR_GoalPricerange.pt"))
model_GoalPricerange.eval()

GoalPricerangeModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_pricerange_classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Print scores

In [21]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.0331074, 0.9887474, 0.3371799, 0.8012731, 0.1714951, 0.8512001)

In [22]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.0268346, 0.991488, 0.2880708, 0.800123, 0.1840107, 0.9121297)

### Train Goal Area Model

In [23]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_area_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalArea = model_GoalArea.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalArea.hidden = model_GoalArea.init_hidden()
        
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalAreaModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_area = model_GoalArea(indices, scores)
            
            loss_goal_area = GOAL_LOSS_FUNCTION(goal_area,
                                                retrieve_gold_GoalArea(raw_Y_train_turn,
                                                                       ontology = ontology,
                                                                       device = DEVICE))
            loss_goal_area.backward(retain_graph = True)
            
            optimizer_GoalAreaModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    goal_area_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_area_early_stopping.wait == 0:
        torch.save(model_GoalArea.state_dict(), "model_PANFMR_GoalArea.pt")
        
    if goal_area_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.1908238(0.19)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9328875(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2006399(0.2)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9465313(0.95)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.1913452(0.19)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9321382(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2094127(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9437196(0.94)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.2064197(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9451021(0.95)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	7/30


INFO:root:DEV Acc:		0.1916058(0.19)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9333786(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2094127(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9452028(0.95)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	8/30


INFO:root:DEV Acc:		0.1947341(0.19)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9324695(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2104448(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9428268(0.94)		0.800123(0.8)		0.9121297(0.91)


### Load Goal Area Model

In [24]:
model_GoalArea = GoalAreaModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_area_dim = GOAL_AREA_DIM,
                               device = DEVICE)

model_GoalArea = model_GoalArea.to(DEVICE)
model_GoalArea.load_state_dict(torch.load("model_PANFMR_GoalArea.pt"))
model_GoalArea.eval()

GoalAreaModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_area_classifier): Linear(in_features=100, out_features=7, bias=True)
)

### Print scores

In [25]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.1952555, 0.9304985, 0.3371799, 0.8012731, 0.1714951, 0.8512001)

In [26]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.2104448, 0.9426566, 0.2880708, 0.800123, 0.1840107, 0.9121297)

### Train Goal Name Model

In [27]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_name_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalName = model_GoalName.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalName.hidden = model_GoalName.init_hidden()
        
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalNameModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_name = model_GoalName(indices, scores)
            
            loss_goal_name = GOAL_LOSS_FUNCTION(goal_name,
                                                retrieve_gold_GoalName(raw_Y_train_turn,
                                                                       ontology = ontology,
                                                                       device = DEVICE))
            loss_goal_name.backward(retain_graph = True)
            
            optimizer_GoalNameModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    goal_name_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_name_early_stopping.wait == 0:
        torch.save(model_GoalName.state_dict(), "model_PANFMR_GoalName.pt")
        
    if goal_name_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.1952555(0.2)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9301937(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2104448(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9427276(0.94)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.1952555(0.2)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.930423(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2104448(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.942823(0.94)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	3/30


INFO:root:DEV Acc:		0.1952555(0.2)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9304209(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2104448(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9428217(0.94)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	4/30


INFO:root:DEV Acc:		0.1952555(0.2)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.9304053(0.93)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.2104448(0.21)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.9428195(0.94)		0.800123(0.8)		0.9121297(0.91)


### Load Goal Name Model

In [28]:
model_GoalName = GoalNameModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_name_dim = GOAL_NAME_DIM,
                               device = DEVICE)

model_GoalName = model_GoalName.to(DEVICE)
model_GoalName.load_state_dict(torch.load("model_PANFMR_GoalName.pt"))
model_GoalName.eval()

GoalNameModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_name_classifier): Linear(in_features=100, out_features=115, bias=True)
)

### Print scores

In [29]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.1952555, 0.9301937, 0.3371799, 0.8012731, 0.1714951, 0.8512001)

In [30]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.2104448, 0.9427276, 0.2880708, 0.800123, 0.1840107, 0.9121297)

### Train Goal Food Model

In [31]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

goal_food_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_GoalFood = model_GoalFood.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_GoalFood.hidden = model_GoalFood.init_hidden()
        
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_GoalFoodModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            goal_food = model_GoalFood(indices, scores)
            
            loss_goal_food = GOAL_LOSS_FUNCTION(goal_food,
                                                retrieve_gold_GoalFood(raw_Y_train_turn,
                                                                       ontology = ontology,
                                                                       device = DEVICE))
            loss_goal_food.backward(retain_graph = True)
            
            optimizer_GoalFoodModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    goal_food_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_goal_accuracy))
    
    if goal_food_early_stopping.wait == 0:
       torch.save(model_GoalFood.state_dict(), "model_PANFMR_GoalFood.pt")
    
    if goal_food_early_stopping.stop_training:
        break

plotly_plot(dev_goal_accuracies, test_goal_accuracies, "Goal")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.5693431(0.57)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.6009741(0.6)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.619259(0.62)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.5465798(0.55)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.6279979(0.63)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.5291764(0.53)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.6627103(0.66)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.4937432(0.49)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6777789(0.68)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.4773619(0.48)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	7/30


INFO:root:DEV Acc:		0.6392075(0.64)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.5195393(0.52)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.6714831(0.67)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.4894845(0.49)		0.800123(0.8)		0.9121297(0.91)
INFO:root:Epoch	8/30


INFO:root:DEV Acc:		0.6347758(0.63)		0.3371799(0.34)		0.1714951(0.17)
INFO:root:DEV L2:		0.5269607(0.53)		0.8012731(0.8)		0.8512001(0.85)


INFO:root:TEST Acc:		0.6716895(0.67)		0.2880708(0.29)		0.1840107(0.18)
INFO:root:TEST L2:		0.4848168(0.48)		0.800123(0.8)		0.9121297(0.91)


### Load Goal Food Model

In [32]:
model_GoalFood = GoalFoodModel(vocabulary_size = VOCABULARY_SIZE,
                               embedding_dim = EMBEDDING_DIM,
                               altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                               hidden_dim = HIDDEN_DIM,
                               goal_food_dim = GOAL_FOOD_DIM,
                               device = DEVICE)

model_GoalFood = model_GoalFood.to(DEVICE)
model_GoalFood.load_state_dict(torch.load("model_PANFMR_GoalFood.pt"))
model_GoalFood.eval()

GoalFoodModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (goal_food_classifier): Linear(in_features=100, out_features=93, bias=True)
)

### Print scores

In [33]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6399896, 0.5130169, 0.3371799, 0.8012731, 0.1714951, 0.8512001)

In [34]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6795335, 0.4684101, 0.2880708, 0.800123, 0.1840107, 0.9121297)

### Train Requested Model

In [35]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

requested_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Requested = model_Requested.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_Requested.hidden = model_Requested.init_hidden()
        
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_RequestedModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            requested = model_Requested(indices, scores)
            
            loss_requested = REQUESTED_LOSS_FUNCTION(requested,
                                                     retrieve_gold_Requested(raw_Y_train_turn,
                                                                             ontology = ontology,
                                                                             device = DEVICE))
            loss_requested.backward(retain_graph = True)
            
            optimizer_RequestedModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                              np.around(test_goal_accuracy, decimals = 2),
                                                              test_requested_accuracy,
                                                              np.around(test_requested_accuracy, decimals = 2),
                                                              test_method_accuracy,
                                                              np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                              np.around(test_goal_l2, decimals = 2),
                                                              test_requested_l2,
                                                              np.around(test_requested_l2, decimals = 2),
                                                              test_method_l2,
                                                              np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    requested_early_stopping.on_epoch_end(epoch = (epoch + 1), current_value = (dev_requested_accuracy))
    
    if requested_early_stopping.wait == 0:
        torch.save(model_Requested.state_dict(), "model_PANFMR_Requested.pt")
        
    if requested_early_stopping.stop_training:
        break

plotly_plot(dev_requested_accuracies, test_requested_accuracies, "Requested")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9578074(0.96)		0.1714951(0.17)
INFO:root:DEV L2:		0.5130169(0.51)		0.8012731(0.8)		0.0831079(0.08)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9657927(0.97)		0.1840107(0.18)
INFO:root:TEST L2:		0.4684101(0.47)		0.800123(0.8)		0.0651543(0.07)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9624955(0.96)		0.1714951(0.17)
INFO:root:DEV L2:		0.5130169(0.51)		0.8012731(0.8)		0.0710907(0.07)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9683973(0.97)		0.1840107(0.18)
INFO:root:TEST L2:		0.4684101(0.47)		0.800123(0.8)		0.0571364(0.06)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6795335(0.68)		0.9741275(0.97)		0.1840107(0.18)
INFO:root:TEST L2:		0.4684101(0.47)		0.800123(0.8)		0.0468269(0.05)
INFO:root:Epoch	10/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9632167(0.96)		0.1714951(0.17)
INFO:root:DEV L2:		0.5130169(0.51)		0.8012731(0.8)		0.0666757(0.07)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9746484(0.97)		0.1840107(0.18)
INFO:root:TEST L2:		0.4684101(0.47)		0.800123(0.8)		0.0444344(0.04)
INFO:root:Epoch	11/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9624955(0.96)		0.1714951(0.17)
INFO:root:DEV L2:		0.5130169(0.51)		0.8012731(0.8)		0.0691453(0.07)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9744747(0.97)		0.1840107(0.18)
INFO:root:TEST L2:		0.4684101(0.47)		0.800123(0.8)		0.0479061(0.05)


### Load Requested Model

In [36]:
model_Requested = RequestedModel(vocabulary_size = VOCABULARY_SIZE,
                                 embedding_dim = EMBEDDING_DIM,
                                 altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                                 hidden_dim = HIDDEN_DIM,
                                 requested_dim = REQUESTED_DIM,
                                 device = DEVICE)

model_Requested = model_Requested.to(DEVICE)
model_Requested.load_state_dict(torch.load("model_PANFMR_Requested.pt"))
model_Requested.eval()

RequestedModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (requested_classifier): Linear(in_features=100, out_features=8, bias=True)
)

### Print scores

In [37]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6399896, 0.5130169, 0.9679048, 0.8012731, 0.1714951, 0.0633283)

In [38]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6795335, 0.4684101, 0.9751693, 0.800123, 0.1840107, 0.0468137)

### Train Method Model

In [39]:
dev_goal_accuracies = []
dev_requested_accuracies = []
dev_method_accuracies = []

test_goal_accuracies = []
test_requested_accuracies = []
test_method_accuracies = []

method_early_stopping = EarlyStopping(patience = 2)

train_indices = np.arange(raw_X_train.shape[0])

for epoch in range(NUM_EPOCHS):
    
    np.random.shuffle(train_indices)
    
    logging.info("Epoch\t{}/{}".format(epoch + 1, NUM_EPOCHS))
    
    model_Method = model_Method.train()
    
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm_notebook(zip(raw_X_train[train_indices], raw_Y_train[train_indices]), total = len(raw_X_train)):

        model_Method.hidden = model_Method.init_hidden()

        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog["turns"], raw_Y_train_dialog["turns"]):
            
            optimizer_MethodModel.zero_grad()
            
            indices, scores = get_index_and_score(raw_X_train_turn, token_to_index, mode = "train", device = DEVICE)
            
            method = model_Method(indices, scores)
            
            loss_method = METHOD_LOSS_FUNCTION(method,
                                               retrieve_gold_Method(raw_Y_train_turn,
                                                                    ontology = ontology,
                                                                    device = DEVICE))
            loss_method.backward(retain_graph = True)
            
            optimizer_MethodModel.step()
    
    dev_goal_accuracy, \
    dev_goal_l2, \
    dev_requested_accuracy, \
    dev_requested_l2, \
    dev_method_accuracy, \
    dev_method_l2 = get_scores(model_GoalPricerange,
                               model_GoalArea,
                               model_GoalName,
                               model_GoalFood,
                               model_Requested,
                               model_Method,
                               raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")
    logging.info("DEV Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_accuracy,
                                                                 np.around(dev_goal_accuracy, decimals = 2),
                                                                 dev_requested_accuracy,
                                                                 np.around(dev_requested_accuracy, decimals = 2),
                                                                 dev_method_accuracy,
                                                                 np.around(dev_method_accuracy, decimals = 2)))
    logging.info("DEV L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(dev_goal_l2,
                                                                np.around(dev_goal_l2, decimals = 2),
                                                                dev_requested_l2,
                                                                np.around(dev_requested_l2, decimals = 2),
                                                                dev_method_l2,
                                                                np.around(dev_method_l2, decimals = 2)))
    dev_goal_accuracies.append(dev_goal_accuracy)
    dev_requested_accuracies.append(dev_requested_accuracy)
    dev_method_accuracies.append(dev_method_accuracy)

    test_goal_accuracy, \
    test_goal_l2, \
    test_requested_accuracy, \
    test_requested_l2, \
    test_method_accuracy, \
    test_method_l2 = get_scores(model_GoalPricerange,
                                model_GoalArea,
                                model_GoalName,
                                model_GoalFood,
                                model_Requested,
                                model_Method,
                                raw_X_test, raw_Y_test, dataset = "dstc2_test")
    logging.info("TEST Acc:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_accuracy,
                                                                  np.around(test_goal_accuracy, decimals = 2),
                                                                  test_requested_accuracy,
                                                                  np.around(test_requested_accuracy, decimals = 2),
                                                                  test_method_accuracy,
                                                                  np.around(test_method_accuracy, decimals = 2)))
    logging.info("TEST L2:\t\t{}({})\t\t{}({})\t\t{}({})".format(test_goal_l2,
                                                                 np.around(test_goal_l2, decimals = 2),
                                                                 test_requested_l2,
                                                                 np.around(test_requested_l2, decimals = 2),
                                                                 test_method_l2,
                                                                 np.around(test_method_l2, decimals = 2)))
    test_goal_accuracies.append(test_goal_accuracy)
    test_requested_accuracies.append(test_requested_accuracy)
    test_method_accuracies.append(test_method_accuracy)
        
    method_early_stopping.on_epoch_end(epoch = (epoch + 1),
                                    current_value = (dev_method_accuracy))
    
    if method_early_stopping.wait == 0:
        torch.save(model_Method.state_dict(), "model_PANFMR_Method.pt")
    
    if method_early_stopping.stop_training:
        break

plotly_plot(dev_method_accuracies, test_method_accuracies, "Method")

INFO:root:Epoch	1/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9679048(0.97)		0.8993792(0.9)
INFO:root:DEV L2:		0.5130169(0.51)		0.1671901(0.17)		0.0633283(0.06)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9751693(0.98)		0.9239532(0.92)
INFO:root:TEST L2:		0.4684101(0.47)		0.1250564(0.13)		0.0468137(0.05)
INFO:root:Epoch	2/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9679048(0.97)		0.8983445(0.9)
INFO:root:DEV L2:		0.5130169(0.51)		0.172788(0.17)		0.0633283(0.06)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9751693(0.98)		0.9273399(0.93)
INFO:root:TEST L2:		0.4684101(0.47)		0.121101(0.12)		0.0468137(0.05)
INFO:root:Epoch	3/30


...

INFO:root:TEST Acc:		0.6795335(0.68)		0.9751693(0.98)		0.9299056(0.93)
INFO:root:TEST L2:		0.4684101(0.47)		0.1150893(0.12)		0.0468137(0.05)
INFO:root:Epoch	5/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9679048(0.97)		0.9048112(0.9)
INFO:root:DEV L2:		0.5130169(0.51)		0.1616559(0.16)		0.0633283(0.06)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9751693(0.98)		0.931445(0.93)
INFO:root:TEST L2:		0.4684101(0.47)		0.1128521(0.11)		0.0468137(0.05)
INFO:root:Epoch	6/30


INFO:root:DEV Acc:		0.6399896(0.64)		0.9679048(0.97)		0.9027419(0.9)
INFO:root:DEV L2:		0.5130169(0.51)		0.1621799(0.16)		0.0633283(0.06)


INFO:root:TEST Acc:		0.6795335(0.68)		0.9751693(0.98)		0.9277504(0.93)
INFO:root:TEST L2:		0.4684101(0.47)		0.116586(0.12)		0.0468137(0.05)


### Load Method Model

In [40]:
model_Method = MethodModel(vocabulary_size = VOCABULARY_SIZE,
                           embedding_dim = EMBEDDING_DIM,
                           altered_embedding_dim = ALTERED_EMBEDDING_DIM,
                           hidden_dim = HIDDEN_DIM,
                           method_dim = METHOD_DIM,
                           device = DEVICE)

model_Method = model_Method.to(DEVICE)
model_Method.load_state_dict(torch.load("model_PANFMR_Method.pt"))
model_Method.eval()

MethodModel(
  (embeddings): Embedding(1149, 170)
  (altered_embeddings): Linear(in_features=171, out_features=300, bias=True)
  (lstm): LSTM(300, 100)
  (method_classifier): Linear(in_features=100, out_features=5, bias=True)
)

### Print scores

In [41]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_dev, raw_Y_dev, dataset = "dstc2_dev")

(0.6399896, 0.5130169, 0.9679048, 0.1589519, 0.9048112, 0.0633283)

In [42]:
get_scores(model_GoalPricerange,
           model_GoalArea,
           model_GoalName,
           model_GoalFood,
           model_Requested,
           model_Method,
           raw_X_test, raw_Y_test, dataset = "dstc2_test")

(0.6795335, 0.4684101, 0.9751693, 0.1139675, 0.9312397, 0.0468137)